In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

import scipy as sc

import sklearn
from sklearn import feature_selection, datasets, model_selection, preprocessing, decomposition, metrics
from sklearn.model_selection import validation_curve, learning_curve, cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm

pd.set_option('display.max_columns', None)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, '../../Libraries_Paper/libraries/Osc_libraries')
import utils

from python_utils import *

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA, KernelPCA
import scipy.io as sio

from sklearn import manifold

from sklearn.preprocessing import MinMaxScaler


### COMMON FUNCTIONS

In [ ]:
def plotTSNE(X,y):
    red = y == 0
    green = y == 1

    plt.figure(figsize=(8,8))
    tsne = manifold.TSNE(n_components=2, init='random', random_state=0, perplexity=30)
    Y = tsne.fit_transform(X)
    principalDf = pd.DataFrame(data = Y,
                               columns = ['pca1', 'pca2'])
    
    class_0 = np.where(y == 0)
    class_1 = np.where(y == 1)
    
    plt.scatter(Y[class_0, 0], Y[class_0, 1], marker='o', color='red')
    plt.scatter(Y[class_1, 0], Y[class_1, 1], marker='x', color='green')
    
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel("1st principal component")
    plt.ylabel("2nd principal component")
    plt.legend(['non-MDR', 'MDR'],prop={'size': 14})
    plt.grid()
    plt.show()
    
    return principalDf

In [ ]:
def normData_minmax (X_train, X_test):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [ ]:
from sklearn.metrics import mean_squared_error

def my_scorer(estimator, X, y=None):
    X_reduced = estimator.transform(X)
    X_preimage = estimator.inverse_transform(X_reduced)
    return -1 * mean_squared_error(X, X_preimage)

def plotKPCA(X, gamma, ncomp):
    kpca = KernelPCA(n_components=ncomp, kernel='rbf', gamma=gamma)
  
    kpca_transform = kpca.fit_transform(X)
    
    autovalores = kpca.lambdas_
    

    explained_variance = np.var(kpca_transform, axis=0)
    ev = explained_variance / np.sum(explained_variance)
    
    plt.figure()
    plt.plot(np.cumsum(ev))
    plt.grid()
    plt.show()

    return autovalores, kpca_transform

def plotPCA(X_train, X_test, emplained_variance):
    pca = PCA(n_components=emplained_variance)

    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
        
    ev = pca.explained_variance_ratio_

    plt.figure()
    plt.plot(np.cumsum(ev))
    plt.grid()
    plt.show()
    
    return X_train_pca,X_test_pca

# PCA

## 1. DTW = DTW_D

In [ ]:
norm = True
folders = ["S1", "S2", "S3", "S4", "S5"]
dependencia = "DTW_D"

for i in range(len(folders)):
    print("-----------------------------------------------------------------------------------")
    print(folders[i])
    print("Balanced data --> " + dependencia)
    X_train = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_train.csv')
    X_test = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_test.csv')

    y_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_train_tensor.csv')
    y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_test_tensor.csv')
    

    if norm:
        X_train, X_test = normData_minmax (X_train, X_test)
        
    gamma = 1
    X_train = np.exp(-gamma*X_train)
    X_test = np.exp(-gamma*X_test)

    emplained_variance = 0.99
    data_train_PCA_1,data_test_PCA_1 = plotPCA(X_train, X_test, emplained_variance)

    plotTSNE(data_train_PCA_1,y_train)
    
    pd.DataFrame(data_train_PCA_1).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/PCA/X_train_Norm_PCA_" + folders[i] + ".csv", index=False)
    pd.DataFrame(data_test_PCA_1).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/PCA/X_test_Norm_PCA_" + folders[i] + ".csv", index=False)
    

##  2. DTW = DTW_I

In [ ]:
norm = True
folders = ["S1", "S2", "S3", "S4", "S5"]
dependencia = "DTW_I"

for i in range(len(folders)):
    print("-----------------------------------------------------------------------------------")
    print(folders[i])
    print("Balanced data --> " + dependencia)
    X_train = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_train.csv')
    X_test = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_test.csv')

    y_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_train_tensor.csv')
    y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_test_tensor.csv')
    

    if norm:
        X_train, X_test = normData_minmax (X_train, X_test)
        
    gamma = 1
    X_train = np.exp(-gamma*X_train)
    X_test = np.exp(-gamma*X_test)

    emplained_variance = 0.99
    data_train_PCA_2,data_test_PCA_2 = plotPCA(X_train, X_test, emplained_variance)

    plotTSNE(data_train_PCA_2,y_train)
    
    pd.DataFrame(data_train_PCA_2).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/PCA/X_train_Norm_PCA_" + folders[i] + ".csv", index=False)
    pd.DataFrame(data_test_PCA_2).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/PCA/X_test_Norm_PCA_" + folders[i] + ".csv", index=False)
    

# KPCA

In [ ]:
from sklearn.metrics import mean_squared_error
def my_scorer(estimator, X, y=None):
    X_reduced = estimator.fit_transform(X)
    X_preimage = estimator.inverse_transform(X_reduced)
    return -1 * mean_squared_error(X, X_preimage)

In [ ]:
def plotTSNE(X,y):
    red = y == 0
    green = y == 1

    plt.figure(figsize=(8,8))
    tsne = manifold.TSNE(n_components=2, init='random', random_state=0, perplexity=30)
    Y = tsne.fit_transform(X)
    principalDf = pd.DataFrame(data = Y,
                               columns = ['pca1', 'pca2'])

    print(principalDf)
    
    class_0 = np.where(y == 0)
    class_1 = np.where(y == 1)

    plt.plot(Y[class_0, 0], Y[class_0, 1], "ko", color='red',mfc='none')
    plt.plot(Y[class_1, 0], Y[class_1, 1], "kx", color='green')
    
    plt.scatter(Y[class_0, 0], Y[class_0, 1], marker='o', color='red')
    plt.scatter(Y[class_1, 0], Y[class_1, 1], marker='x', color='green')
    
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlabel("1st principal component")
    plt.ylabel("2nd principal component")
    plt.legend(['non-MDR', 'MDR'],prop={'size': 14})
    plt.grid()
    plt.show()
    
    return principalDf

## 1. DTW = DTW_D

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error


folders = ["S1", "S2", "S3", "S4", "S5"]

semillas = [9,18,35, 52, 75]
norm = True
dependencia = "DTW_D"

for i in range(len(folders)):
    print("-----------------------------------------------------------------------------------")
    print(folders[i])

    X_train = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_train.csv')
    X_test = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_test.csv')

    y_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_train_tensor.csv')
    y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_test_tensor.csv')
    
    
    param_grid = [{
        "kernel": ["rbf", "sigmoid"],
        "n_components" : [20, 30, 35, 40, 45, 50, 60, 70],
        "alpha": [.0005, .001] 
    }]

    
    if norm:
        X_train_pre, X_test = normData_minmax (X_train, X_test)


    kpca = KernelPCA(fit_inverse_transform=True, n_jobs=-1) 
    grid_search_1 = GridSearchCV(kpca, param_grid, cv=5, scoring=my_scorer)
    grid_search_1.fit(X_train_pre)


    print("========>", grid_search_1.best_params_)
    
    kpca_aux_1 = KernelPCA(n_components=grid_search_1.best_params_['n_components'],
                     kernel=grid_search_1.best_params_['kernel'],
                     n_jobs=-1) 
    data_train_KPCA_1 = kpca_aux_1.fit_transform(X_train_pre)
    data_test_KPCA_1 = kpca_aux_1.transform(X_test)
    

    plotTSNE(data_train_KPCA_1,y_train)
    
    pd.DataFrame(data_train_KPCA_1).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/KPCA/X_train_Norm_KPCA_" + folders[i] + ".csv", index=False)
    pd.DataFrame(data_test_KPCA_1).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/KPCA/X_test_Norm_KPCA_" + folders[i] + ".csv", index=False)
    

## 2. DTW = DTW_I

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error


folders = ["S1", "S2", "S3", "S4", "S5"]

semillas = [9,18,35, 52, 75]
norm = True
dependencia = "DTW_I"

for i in range(len(folders)):
    print("-----------------------------------------------------------------------------------")
    print(folders[i])

    X_train = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_train.csv')
    X_test = pd.read_csv('../data_generated_by_dtw/' + dependencia + '/' + folders[i] + '/X_test.csv')

    y_train = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_train_tensor.csv')
    y_test = pd.read_csv('../../df_to_load/DataToPaperAndTFM_Mod1/Otras normalizaciones/Subconjuntos_3D_norm/' + folders[i] + '/y_test_tensor.csv')
    
    
    param_grid = [{
        "kernel": ["rbf", "sigmoid"],
        "n_components" : [20, 30, 40, 50, 60, 70, 80, 90],
        "alpha": [.0005, .001] 
    }]

    
    if norm:
        X_train_pre, X_test = normData_minmax (X_train, X_test)


    kpca = KernelPCA(fit_inverse_transform=True, n_jobs=-1) 
    grid_search_2 = GridSearchCV(kpca, param_grid, cv=5, scoring=my_scorer)
    grid_search_2.fit(X_train_pre)


    print("========>", grid_search_2.best_params_)
    
    kpca_aux_2 = KernelPCA(n_components=grid_search_2.best_params_['n_components'],
                     kernel=grid_search_2.best_params_['kernel'],
                     n_jobs=-1) 
    
    data_train_KPCA_2 = kpca_aux_2.fit_transform(X_train_pre)
    data_test_KPCA_2 = kpca_aux_2.transform(X_test)
    

    plotTSNE(data_train_KPCA_2,y_train)
    
    pd.DataFrame(data_train_KPCA_2).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/KPCA/X_train_Norm_KPCA_" + folders[i] + ".csv", index=False)
    pd.DataFrame(data_test_KPCA_2).to_csv("../1_Clasifications_models/data_reduced/" + dependencia + "/KPCA/X_test_Norm_KPCA_" + folders[i] + ".csv", index=False)
    